In [9]:
import numpy as np
from PySDM import Formulae
from PySDM.physics import si
from PySDM.initialisation.spectra import Lognormal
from PySDM.products import (
    ParcelDisplacement, AmbientTemperature, AmbientDryAirDensity, AmbientRelativeHumidity,
    ParticleSizeSpectrumPerVolume, ParticleVolumeVersusRadiusLogarithmSpectrum
)

from PySDM_examples.Pyrcel import Settings, Simulation

In [106]:
settings = Settings(
    dz = 10 * si.m,
    n_sd_per_mode = (1, 1),
    aerosol_modes_by_kappa = {
        .54: Lognormal(
            norm_factor=850 / si.cm ** 3,
            m_mode=15 * si.nm,
            s_geom=1.6
        ),
        1.2: Lognormal(
            norm_factor=10 / si.cm ** 3,
            m_mode=850 * si.nm,
            s_geom=1.2
        )
    },
    vertical_velocity = 1.0 * si.m / si.s,
    initial_pressure = 775 * si.mbar,
    initial_temperature = 274 * si.K,
    initial_relative_humidity = .98,
    displacement = 1000 * si.m,
    formulae = Formulae(constants={'MAC': .3})
)

dry_radius_bin_edges = np.logspace(
    np.log10(1e-3 * si.um),
    np.log10(5e0 * si.um),
    33, endpoint=False
)

simulation = Simulation(settings, products=(
    ParcelDisplacement(
        name='z'),
    AmbientRelativeHumidity(
        name='S_max_percent', unit='%', var='RH'),
    AmbientTemperature(
        name='T'),
    ParticleSizeSpectrumPerVolume(
        name='dry:dN/dR', radius_bins_edges=dry_radius_bin_edges, dry=True),
    ParticleVolumeVersusRadiusLogarithmSpectrum(
        name='dry:dV/dlnR', radius_bins_edges=dry_radius_bin_edges, dry=True),
    AmbientDryAirDensity(),
))

In [107]:
output = simulation.run()

In [108]:
from pyevtk.hl import unstructuredGridToVTK, pointsToVTK
from pyevtk.vtk import VtkHexahedron, VtkGroup
import numpy as np

from PySDM.exporters import VTKExporter


class VTKExporterParcel(VTKExporter):
    def __init__(self, n_sd):
        super().__init__()
        self.x = np.random.random(n_sd)
        self.y = np.random.random(n_sd)
        self.z = np.random.random(n_sd)

    def write_pvd(self):
        pvd_attributes = VtkGroup(self.attributes_file_path)
        for k, v in self.exported_times["attributes"].items():
            pvd_attributes.addFile(k + ".vtu", sim_time=v)
        pvd_attributes.save()

        pvd_products = VtkGroup(self.products_file_path)
        for k, v in self.exported_times["products"].items():
            pvd_products.addFile(k + ".vtu", sim_time=v)
        pvd_products.save()

    def export_products(self, output, step):
        path = (
            self.products_file_path
            + "_num"
            + self.add_leading_zeros(step)
        )
        print(path)
        self.exported_times["products"][path] = (
            step * simulation.particulator.dt
        )

        n_levels = len(output['products']['z'])
        n_vertices = 4 * n_levels
        x = np.zeros(n_vertices)
        y = np.zeros(n_vertices)
        z = np.zeros(n_vertices)
        conn = [0, 1, 2, 3]
        ctype = np.zeros(n_levels - 1)
        ctype[:] = VtkHexahedron.tid

        _volume = simulation.particulator.environment.mass_of_dry_air / output['products']['rhod'][0]
        _dz = output['products']['z'][1] - output['products']['z'][0]
        for level in range(n_levels): 
            _z = output['products']['z'][level] 
            if level > 0:
                prev_to_curr_density_ratio = output['products']['rhod'][level-1] / output['products']['rhod'][level]
                _volume *= prev_to_curr_density_ratio
                _dz = output['products']['z'][level] - output['products']['z'][level-1]
            _area = _volume / _dz
            _half_diagonal = (2 * _area) ** .5
            x[i := level * 4], y[i], z[i] = -_half_diagonal, -_half_diagonal, _z 
            x[i := i + 1], y[i], z[i] = -_half_diagonal, _half_diagonal, _z
            x[i := i + 1], y[i], z[i] = _half_diagonal, _half_diagonal, _z
            x[i := i + 1], y[i], z[i] = _half_diagonal, -_half_diagonal, _z
            conn += [*range(4 * (level + 1), 4 * (level + 2))] * 2
        conn = np.asarray(conn[:-4])
        offset = np.asarray(range(8, 8 * n_levels, 8))

        point_data = {"test_pd": np.array([44] * n_vertices)}

        _RH = output['products']['S_max_percent']
        cell_data = {"RH": np.full(shape=(len(_RH)-1,), fill_value=np.nan)}
        cell_data["RH"][:step] = (np.array(_RH[:-1] + np.diff(_RH)/2))[:step]
        unstructuredGridToVTK(
            path,
            x,
            y,
            z,
            connectivity=conn,
            offsets=offset,
            cell_types=ctype,
            cellData=cell_data,
            #pointData=point_data,
            #fieldData=field_data,
        )

    def export_attributes(self, output, step):
        path = (
            self.attributes_file_path
            + "_num"
            + self.add_leading_zeros(step)
        )
        self.exported_times["attributes"][path] = step * simulation.particulator.dt
        print(path)
        payload = {}

        for k in output["attributes"].keys():
            payload[k] = np.asarray(output["attributes"][k])[:,step].copy()

        if step != 0:
            dz = output['products']['z'][step] - output['products']['z'][step - 1] 
            if step == 1:
                self.z *= dz
            else:
                self.z += dz
        print(self.z)

        pointsToVTK(path, self.x, self.y, self.z, data=payload)

In [109]:
e = VTKExporterParcel(n_sd=simulation.particulator.n_sd)
for step in settings.output_steps:
    e.export_products(output, step)
    e.export_attributes(output, step)
e.write_pvd()

.\output\sd_products_num0000000000
.\output\sd_attributes_num0000000000
[0.46144518 0.47033488]
.\output\sd_products_num0000000001
.\output\sd_attributes_num0000000001
[4.61445177 4.70334881]
.\output\sd_products_num0000000002
.\output\sd_attributes_num0000000002
[14.61445177 14.70334881]
.\output\sd_products_num0000000003
.\output\sd_attributes_num0000000003
[24.61445177 24.70334881]
.\output\sd_products_num0000000004
.\output\sd_attributes_num0000000004
[34.61445177 34.70334881]
.\output\sd_products_num0000000005
.\output\sd_attributes_num0000000005
[44.61445177 44.70334881]
.\output\sd_products_num0000000006
.\output\sd_attributes_num0000000006
[54.61445177 54.70334881]
.\output\sd_products_num0000000007
.\output\sd_attributes_num0000000007
[64.61445177 64.70334881]
.\output\sd_products_num0000000008
.\output\sd_attributes_num0000000008
[74.61445177 74.70334881]
.\output\sd_products_num0000000009
.\output\sd_attributes_num0000000009
[84.61445177 84.70334881]
.\output\sd_products_num

In [96]:
!dir output

 Volume in drive C is OS
 Volume Serial Number is 4AFE-B6B9

 Directory of c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output

11.08.2025  18:45    <DIR>          .
11.08.2025  17:25    <DIR>          ..
11.08.2025  18:45             9�510 sd_attributes.pvd
11.08.2025  18:45            11�368 sd_attributes_num0000000000.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000001.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000002.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000003.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000004.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000005.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000006.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000007.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000008.vtu
11.08.2025  18:45            11�368 sd_attributes_num0000000009.vtu
11.08.2025  18:45            11�368 sd_attribu